In [10]:
import numpy as np
import pandas as pd
import pickle
from google.cloud import storage,bigquery
from pandas_gbq import to_gbq
import requests

In [11]:
def load_modele_from_bucket(modele_url):
    bucket_name, object_path = modele_url.split('gs://')[1].split('/', 1)
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(object_path)
    # Téléchargez le contenu de l'objet
    modele_content = blob.download_as_bytes()
    return modele_content

In [12]:
clients = storage.Client()

In [19]:
bucket_name = 'BUCKET_NAME'
file_name = 'FILE_PATH'
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)
blob.download_to_filename('FILE_PATH')
df = pd.read_csv('FILE_PATH', sep=',')

In [20]:
df

,valeur,Latitude,Longitude,type d'implantation,type d'influence,Year,Month,Day
0,0.0,49.119442,6.180833,1,1,2022,8,1
1,7.0,49.110283,6.223336,1,1,2022,8,1
2,10.0,48.915000,6.058336,1,1,2022,8,1
3,7.0,49.368610,6.160831,1,1,2022,8,1
4,11.0,49.358337,6.156942,1,1,2022,8,1
...,...,...,...,...,...,...,...,...
226115,10.0,47.294640,6.825861,5,1,2023,9,30
226116,14.0,47.096720,5.496389,1,1,2023,9,30
226117,12.0,47.147250,5.556117,2,2,2023,9,30
226118,12.0,47.620022,6.157642,1,1,2023,9,30


In [21]:
model_url = 'FILE_PATH'
modele_content=load_modele_from_bucket(model_url)
RF = pickle.loads(modele_content)

In [22]:
x=df.drop(['valeur'],axis=1)

In [24]:
y_predicted = RF.predict(x)

In [27]:
Scores = pd.DataFrame({'Year': df['Year'], 'Month': df['Month'],'Day':  df['Day'],'valeur':  df['valeur'],'Latitude':  df['Latitude'],'Longitude':  df['Longitude'], 'prediction': y_predicted})

In [28]:
Scores

,Year,Month,Day,valeur,Latitude,Longitude,prediction
0,2022,8,1,0.0,49.119442,6.180833,4.09
1,2022,8,1,7.0,49.110283,6.223336,6.53
2,2022,8,1,10.0,48.915000,6.058336,8.94
3,2022,8,1,7.0,49.368610,6.160831,7.44
4,2022,8,1,11.0,49.358337,6.156942,9.73
...,...,...,...,...,...,...,...
226115,2023,9,30,10.0,47.294640,6.825861,9.92
226116,2023,9,30,14.0,47.096720,5.496389,13.51
226117,2023,9,30,12.0,47.147250,5.556117,13.24
226118,2023,9,30,12.0,47.620022,6.157642,11.86


In [31]:
gfg_csv_data = Scores.to_csv('FILE_PATH', index = False)